## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
503
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+--------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|               res_2|               res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+--------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
| -4673.18141230324|-2309.3414111954507|-143.7924205220715|   2505.5|SSSSBBBB| 35.9478|-106.7469|       SNWD|  0.220473104555138|0.045165507547204914| 0.04577000867740661|0.894

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year      coeff_2
0  USC00299820  1984.0 -2309.341411
1  USC00299820  1985.0  -831.394998
2  USC00298011  1968.0  2425.790641
3  USC00299820  1979.0 -1382.988673
4  USC00298011  1960.0  -699.628566

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  US1NMBR0034  USC00290225  USC00290743  USC00290903  USC00291080  \
year                                                                       
1948.0           NaN          NaN  -147.348495          NaN          NaN   
1949.0           NaN          NaN  -256.331449          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN   -26.494831          NaN          NaN   
1952.0           NaN          NaN     7.046026          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1956.0           NaN          NaN    -8.994599          NaN     6.133287   
1957.0           NaN          NaN          NaN          NaN          NaN   

station  USC00291086  USC00291647  USC00291982  USC00292100  USC00292139  \
year                                                                       
1948.0     92.414234          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN          NaN          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN    82.543097          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN   -72.462037   
1956.0           NaN          NaN          NaN          NaN  -282.876221   
1957.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00298830  USC00298834  USC00299031  USC00299113  \
year        ...                                                            
1948.0      ...               NaN          NaN          NaN   -16.032611   
1949.0      ...               NaN          NaN          NaN          NaN   
1950.0      ...               NaN          NaN          NaN          NaN   
1951.0      ...               NaN          NaN          NaN          NaN   
1952.0      ...               NaN          NaN          NaN          NaN   
1953.0      ...               NaN          NaN          NaN          NaN   
1954.0      ...               NaN          NaN          NaN          NaN   
1955.0      ...               NaN          NaN          NaN          NaN   
1956.0      ...        -50.350314          NaN          NaN          NaN   
1957.0      ...               NaN          NaN          NaN          NaN   

station  USC00299193  USC00299496  USC00299820  USW00023049  USW00023054  \
year                                                                       
1948.0           NaN          NaN          NaN          NaN    77.553174   
1949.0           NaN          NaN          NaN   -31.578275    -9.399484   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN    39.637198   
1953.0           NaN          NaN  -271.133493          NaN    41.333440   
1954.0           NaN          NaN          NaN          NaN   -32.798985   
1955.0           NaN          NaN  -356.979997          NaN          NaN   
1956.0     20.546444          NaN  -554.706544          NaN          NaN   
1957.0           NaN          NaN   -42.953996          NaN          NaN   

station  USW00093057  
year                  
1948.0           NaN  
1949.0           NaN  
1950.0           NaN  
1951.0           NaN  
1952.0           NaN  
1953.0           NaN  
1954.0           NaN  
1955.0           NaN  
1956.0           NaN  
1957.0           NaN  

[10 rows x 52 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [10]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  564.056321801
RMS removing mean-by-station=  514.466414579
RMS removing mean-by-year   =  459.995375319


In [11]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 564.056321801
0 after removing mean by year    = 459.995375319
0 after removing mean by stations= 421.6923599
1 after removing mean by year    = 418.761677543
1 after removing mean by stations= 418.348392256
2 after removing mean by year    = 418.268206543
2 after removing mean by stations= 418.249818958
3 after removing mean by year    = 418.244963997
3 after removing mean by stations= 418.243466849
4 after removing mean by year    = 418.242922481
4 after removing mean by stations= 418.242693213
